In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = 'bert-base-uncased'
raw_dataset = load_dataset('glue', 'mrpc')
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example['sentence1'], example['sentence2'],
                     truncation=True)

tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Specify training arguments hyperparameters:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=5,
                                  learning_rate=2e-5,
                                  weight_decay=0.01)

# Create the Trainer instance:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer
)
trainer.train()

'''The results will however be anticlimatic because 
you will only get a training loss that doesn't tell you how well the
model is actually doing..
To fix this, you need to specify the evaluation metric'''

predictions = trainer.predict(tokenized_dataset['validation'])
print(predictions)
print(predictions.predictions.shape, predictions.label_ids.shape)

# it returns a named tuple with 3 elements: predictions, label_ids, metrics
# the predictions are the logits of the model with all the sentences of the dataset
# so a numpy array of shape(488 x 2)

# to match them with our labels we need to take the maximum logits for each prediction
# to know which is the maximum, use the argmax function
import numpy as np
from datasets import load_metric

metric = load_metric('glue', 'mrpc')
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

'''We can see that our model did learn something!'''

In [ ]:
'''To monitor the metrics during training, we need to define a
compute metric function as we did above
and pass it to the Trainer
'''
metric = load_metric('glue','mrpc')
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments("test-trainer",
                                  evaluation_strategy='epoch')
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
